In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls

In [ ]:
import pandas as pd
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt


import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from sklearn.utils import shuffle


## Creating Dataset

In [ ]:
path_withmask = glob.glob('/kaggle/input/face-mask-detection/Dataset/with_mask/*')
path_withoutmask = glob.glob('/kaggle/input/face-mask-detection/Dataset/without_mask/*')
path_maskincorred = glob.glob('/kaggle/input/face-mask-detection/Dataset/mask_weared_incorrect/*')

In [ ]:
def getimage(path):
    img = cv2.imread(path)
    resized = cv2.resize(img,(224,224))
    return resized

In [ ]:
mask_detector_dict = {
    "withmask" : path_withmask,
    "withoutmask" : path_withoutmask,
    "maskweared_incorrect" : path_maskincorred
}

In [ ]:
mask_detector_values = {
    "withmask" : 0,
    "withoutmask" : 1,
    "maskweared_incorrect" : 2
}

In [ ]:
 X,y = [],[]

for mask_type,mask_data in mask_detector_dict.items():
    for item in mask_data:
        X.append(item)
        y.append(mask_detector_values[mask_type])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
data= {
    "img" : X,
    "type" : y
}

df = pd.DataFrame(data)

In [ ]:
df = shuffle(df)
df = df.dropna()

In [ ]:
X = df['img']
y = df['type']

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Creating Model

In [ ]:
feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224,224,3),
    trainable=False
)

In [ ]:
num_of_types = 3

model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    layers.Dense(num_of_types)
])

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)



### Training Model

In [ ]:
def batch_train(X,y,batch):
    n = len(X)
    pos = 0
    count = 1
    while pos < n:
        print(f'Batch {count}')
        if (pos + batch) > n:
            batch = n-pos
        
        train_X = X[pos:pos+batch]
        train_Y = y[pos:pos+batch]
        
        X_batch = []
        
        for item in train_X:
            X_batch.append(getimage(item))
            
        
        X_batch = np.array(X_batch)
        train_Y = np.array(train_Y)
        X_batch = X_batch/255
        print(train_Y)
        model.fit(X_batch,train_Y, epochs = 5)
        pos +=batch
        count = count+1

In [ ]:
batch_train(X_train,y_train,300)

In [ ]:
len(X_test)

### Evaluating Model

In [ ]:
def batch_evaluate(X,y,batch):
    pos = 0
    count = 0
    
    n = len(X)
    
    print(n)
    
    count = 1
    no_of_batches  = n/batch
    accuracy = 0
    while pos < n:
        print(f'Batch {count} of {int(no_of_batches+1)}')
        if pos+batch > n:
            batch  = n-pos
        
        test_X = X[pos:pos+batch]
        test_Y = y[pos:pos+batch]
        
        X_batch = []
        
        for item in test_X:
            X_batch.append(getimage(item))

        X_batch = np.array(X_batch)
        test_Y = np.array(test_Y)
        X_batch = X_batch/255
        
        result = model.evaluate(X_batch,test_Y)
        print(result)
        pos+=batch
        accuracy += result[1]
        count = count+1
        
    print(f'Accuracy : {accuracy/count}')

In [ ]:
batch_evaluate(X_test,y_test,300)

In [ ]:
model.save('./model.h5')

In [ ]:
model.save_weights('modelweights.h5')